In [ ]:
# Step 1: Install Kaggle and set up authentication
!pip install kaggle

# Step 2: Upload your kaggle.json file from your local machine.
# This will prompt you to select the kaggle.json file.
from google.colab import files
files.upload()  # upload your kaggle.json file here

# Step 3: Move kaggle.json into the correct directory and set permissions.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the dataset from Kaggle.
# Replace <dataset-identifier> with the actual Kaggle dataset identifier.
# For example, if the dataset URL is https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset,
# then the dataset identifier is vipoooool/new-plant-diseases-dataset.
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset -p /content/data --unzip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import plot_model

In [ ]:


# Data directories and parameters
data_dir = '/content/data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'
img_size = (224, 224)
batch_size = 32

# Validation data generator (for confusion matrices)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_generator = datagen.flow_from_directory(
    data_dir + '/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # Ensures predictions align with true labels
)





In [ ]:
# Load your trained models
models = {
    'ResNet50': load_model('/content/ResNet50.h5'),
    'VGG19': load_model('/content/VGG19.h5'),
    'InceptionV3': load_model('/content/InceptionV3.h5'),
    'MobileNetV2': load_model('/content/MobileNetV2.h5'),
    'DenseNet121': load_model('/content/DenseNet121.h5')
}

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(model, model_name):
    try:
        # Get true labels and predictions
        y_true = val_generator.classes
        y_pred = np.argmax(model.predict(val_generator), axis=1)
    except TypeError as e:
        print(f"Error during prediction for {model_name}: {e}")
        return
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Plot
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=val_generator.class_indices.keys(),
                yticklabels=val_generator.class_indices.keys())
    plt.title(f"{model_name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

# Plot confusion matrices for all models
print("Generating confusion matrices...")
for model_name, model in models.items():
    plot_confusion_matrix(model, model_name)

# Plot and save model diagrams
print("\nGenerating model diagrams...")
for model_name, model in models.items():
    output_file = f"{model_name.lower()}_model.png"
    print(f"Saving {model_name} diagram to {output_file}")
    plot_model(model, to_file=output_file, show_shapes=True, show_layer_names=True)

In [ ]:


# Plot confusion matrices for all models
print("Generating confusion matrices...")
for model_name, model in models.items():
    plot_confusion_matrix(model, model_name)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import plot_model
# Import the patches module from matplotlib
import matplotlib.patches as patches # Added this line to import the patches module

In [ ]:
def plot_simplified_model(model, model_name):
    """
    Generates a simplified high-level block diagram of the model.
    Groups layers into blocks like 'Convolutional Block', 'Pooling Layer', etc.
    """
    blocks = []
    current_block = None

    # Analyze layers and group into blocks
    for layer in model.layers:
        layer_type = layer.__class__.__name__
        if layer_type == 'Conv2D':
            if current_block != 'Convolutional Block':
                blocks.append('Convolutional Block')
                current_block = 'Convolutional Block'
        elif layer_type in ['MaxPooling2D', 'AveragePooling2D']:
            blocks.append('Pooling Layer')
            current_block = 'Pooling Layer'
        elif layer_type == 'Flatten':
            blocks.append('Flatten Layer')
            current_block = 'Flatten Layer'
        elif layer_type == 'Dense':
            if current_block != 'Dense Block':
                blocks.append('Dense Block')
                current_block = 'Dense Block'

    # Add Input and Output blocks
    blocks.insert(0, 'Input Layer')
    blocks.append('Output')

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.set_xlim(0, len(blocks))
    ax.set_ylim(0, 1)
    ax.axis('off')

    # Draw blocks as rectangles
    for i, block in enumerate(blocks):
        rect = patches.Rectangle((i, 0.25), 0.8, 0.5, linewidth=1, edgecolor='black', facecolor='lightblue')
        ax.add_patch(rect)
        ax.text(i + 0.4, 0.5, block, ha='center', va='center', fontsize=10)

    # Draw arrows between blocks
    for i in range(len(blocks) - 1):
        ax.arrow(i + 0.8, 0.5, 0.4, 0, head_width=0.05, head_length=0.1, fc='black', ec='black')

    plt.title(f'Simplified {model_name} Architecture')
    plt.savefig(f'simplified_{model_name.lower()}.png')
    plt.show()

In [ ]:
# Plot and save model diagrams (both detailed and simplified)
print("\nGenerating model diagrams...")
for model_name, model in models.items():
    # Detailed diagram
    output_file = f"{model_name.lower()}_model.png"
    print(f"Saving detailed {model_name} diagram to {output_file}")
    plot_model(model, to_file=output_file, show_shapes=True, show_layer_names=True)

    # Simplified diagram
    print(f"Generating simplified diagram for {model_name}")
    plot_simplified_model(model, model_name)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_block(ax, x, y, width, height, text, color='lightblue'):
    """Draws a single block with text."""
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='black', facecolor=color)
    ax.add_patch(rect)
    ax.text(x + width / 2, y + height / 2, text, ha='center', va='center', fontsize=8, wrap=True)

def draw_arrow(ax, start_x, start_y, end_x, end_y):
    """Draws an arrow between two points."""
    ax.arrow(start_x, start_y, end_x - start_x, end_y - start_y, head_width=0.1, head_length=0.1, fc='black', ec='black')

def plot_model_architecture(model_name, blocks, figsize=(20, 10), output_file=None):
    """Generic function to plot model architecture."""
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(0, len(blocks))
    ax.set_ylim(0, 1)
    ax.axis('off')

    # Draw blocks and arrows
    for i, block in enumerate(blocks):
        draw_block(ax, i, 0.25, 0.8, 0.5, block['name'], block['color'])
        if i < len(blocks) - 1:
            draw_arrow(ax, i + 0.8, 0.5, i + 1, 0.5)

    plt.title(f'{model_name} Architecture', fontsize=16)
    if output_file:
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.show()

# Define blocks for each model
def get_resnet50_blocks():
    """Returns blocks for ResNet50 architecture."""
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Conv1\n112x112x64', 'color': 'lightgreen'},
        {'name': 'Pool1\n56x56x64', 'color': 'lightpink'},
        {'name': 'Conv2_x\n56x56x256', 'color': 'lightcoral'},
        {'name': 'Conv3_x\n28x28x512', 'color': 'lightcoral'},
        {'name': 'Conv4_x\n14x14x1024', 'color': 'lightcoral'},
        {'name': 'Conv5_x\n7x7x2048', 'color': 'lightcoral'},
        {'name': 'Global Avg Pool\n1x1x2048', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ]

def get_vgg19_blocks():
    """Returns blocks for VGG19 architecture."""
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Block1\n224x224x64', 'color': 'lightgreen'},
        {'name': 'Block2\n112x112x128', 'color': 'lightgreen'},
        {'name': 'Block3\n56x56x256', 'color': 'lightgreen'},
        {'name': 'Block4\n28x28x512', 'color': 'lightgreen'},
        {'name': 'Block5\n14x14x512', 'color': 'lightgreen'},
        {'name': 'Flatten\n25088', 'color': 'lightyellow'},
        {'name': 'FC1\n4096', 'color': 'lightblue'},
        {'name': 'FC2\n4096', 'color': 'lightblue'},
        {'name': 'FC3\n1000', 'color': 'lightblue'}
    ]

def get_inceptionv3_blocks():
    """Returns blocks for InceptionV3 architecture."""
    return [
        {'name': 'Input\n299x299x3', 'color': 'lightblue'},
        {'name': 'Stem\n35x35x384', 'color': 'lightgreen'},
        {'name': 'Inception A\n35x35x256', 'color': 'lightcoral'},
        {'name': 'Inception B\n35x35x288', 'color': 'lightcoral'},
        {'name': 'Inception C\n35x35x288', 'color': 'lightcoral'},
        {'name': 'Reduction A\n17x17x768', 'color': 'lightyellow'},
        {'name': 'Inception D\n17x17x768', 'color': 'lightcoral'},
        {'name': 'Reduction B\n8x8x1280', 'color': 'lightyellow'},
        {'name': 'Inception E\n8x8x2048', 'color': 'lightcoral'},
        {'name': 'Global Avg Pool\n1x1x2048', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ]

def get_mobilenetv2_blocks():
    """Returns blocks for MobileNetV2 architecture."""
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Conv1\n112x112x32', 'color': 'lightgreen'},
        {'name': 'Bottleneck 1\n112x112x16', 'color': 'lightcoral'},
        {'name': 'Bottleneck 2\n56x56x24', 'color': 'lightcoral'},
        {'name': 'Bottleneck 3\n28x28x32', 'color': 'lightcoral'},
        {'name': 'Bottleneck 4\n14x14x64', 'color': 'lightcoral'},
        {'name': 'Bottleneck 5\n14x14x96', 'color': 'lightcoral'},
        {'name': 'Bottleneck 6\n7x7x160', 'color': 'lightcoral'},
        {'name': 'Bottleneck 7\n7x7x320', 'color': 'lightcoral'},
        {'name': 'Conv2\n7x7x1280', 'color': 'lightgreen'},
        {'name': 'Global Avg Pool\n1x1x1280', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ]

def get_densenet121_blocks():
    """Returns blocks for DenseNet121 architecture."""
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Conv1\n112x112x64', 'color': 'lightgreen'},
        {'name': 'Pool1\n56x56x64', 'color': 'lightpink'},
        {'name': 'Dense Block 1\n56x56x256', 'color': 'lightcoral'},
        {'name': 'Transition 1\n28x28x128', 'color': 'lightyellow'},
        {'name': 'Dense Block 2\n28x28x512', 'color': 'lightcoral'},
        {'name': 'Transition 2\n14x14x256', 'color': 'lightyellow'},
        {'name': 'Dense Block 3\n14x14x1024', 'color': 'lightcoral'},
        {'name': 'Transition 3\n7x7x512', 'color': 'lightyellow'},
        {'name': 'Dense Block 4\n7x7x1024', 'color': 'lightcoral'},
        {'name': 'Global Avg Pool\n1x1x1024', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ]

# Generate diagrams for all models
models_blocks = {
    'ResNet50': get_resnet50_blocks(),
    'VGG19': get_vgg19_blocks(),
    'InceptionV3': get_inceptionv3_blocks(),
    'MobileNetV2': get_mobilenetv2_blocks(),
    'DenseNet121': get_densenet121_blocks()
}

for model_name, blocks in models_blocks.items():
    output_file = f"{model_name.lower()}_architecture.png"
    plot_model_architecture(model_name, blocks, output_file=output_file)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_block(ax, x, y, width, height, text, color='lightblue'):
    """Draws a single block with text."""
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='black', facecolor=color)
    ax.add_patch(rect)
    ax.text(x + width / 2, y + height / 2, text, ha='center', va='center', fontsize=8, wrap=True)

def draw_arrow(ax, start_x, start_y, end_x, end_y):
    """Draws an arrow between two points."""
    ax.arrow(start_x, start_y, end_x - start_x, end_y - start_y, head_width=0.1, head_length=0.1, fc='black', ec='black')

def plot_model_architecture(model_name, blocks, explanation, figsize=(20, 12), output_file=None):
    """Generic function to plot model architecture with explanation and legend."""
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(0, len(blocks) + 2)  # Extra space for legend and explanation
    ax.set_ylim(0, 1.5)  # Extra space for explanation at the bottom
    ax.set_xlabel('Data Flow (from Input to Output)', fontsize=10)
    ax.set_ylabel('Layer Stages', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.3)  # Light grid for reference

    # Draw blocks and arrows
    for i, block in enumerate(blocks):
        draw_block(ax, i, 0.25, 0.8, 0.5, block['name'], block['color'])
        if i < len(blocks) - 1:
            draw_arrow(ax, i + 0.8, 0.5, i + 1, 0.5)

    # Add title
    plt.title(f'{model_name} Architecture', fontsize=16, pad=20)

    # Add legend
    legend_elements = [
        patches.Patch(facecolor='lightblue', edgecolor='black', label='Input/Output'),
        patches.Patch(facecolor='lightgreen', edgecolor='black', label='Convolutional'),
        patches.Patch(facecolor='lightpink', edgecolor='black', label='Pooling'),
        patches.Patch(facecolor='lightcoral', edgecolor='black', label='Model-Specific Blocks'),
        patches.Patch(facecolor='lightyellow', edgecolor='black', label='Transition/Reduction')
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=8)

    # Add explanation at the bottom
    explanation_text = f"\nExplanation:\n{explanation}"
    ax.text(len(blocks) / 2, 0.1, explanation_text, ha='center', va='top', fontsize=8, bbox=dict(facecolor='white', alpha=0.8))

    # Save and display the diagram
    if output_file:
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.show()

# Define blocks and explanations for each model
def get_resnet50_blocks():
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Conv1\n112x112x64', 'color': 'lightgreen'},
        {'name': 'Pool1\n56x56x64', 'color': 'lightpink'},
        {'name': 'Conv2_x\n56x56x256', 'color': 'lightcoral'},
        {'name': 'Conv3_x\n28x28x512', 'color': 'lightcoral'},
        {'name': 'Conv4_x\n14x14x1024', 'color': 'lightcoral'},
        {'name': 'Conv5_x\n7x7x2048', 'color': 'lightcoral'},
        {'name': 'Global Avg Pool\n1x1x2048', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ], "ResNet50 uses residual connections to allow deeper networks by adding skip connections, with 50 layers organized into 5 stages of convolutional blocks, ending in a global average pooling and fully connected layer."

def get_vgg19_blocks():
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Block1\n224x224x64', 'color': 'lightgreen'},
        {'name': 'Block2\n112x112x128', 'color': 'lightgreen'},
        {'name': 'Block3\n56x56x256', 'color': 'lightgreen'},
        {'name': 'Block4\n28x28x512', 'color': 'lightgreen'},
        {'name': 'Block5\n14x14x512', 'color': 'lightgreen'},
        {'name': 'Flatten\n25088', 'color': 'lightyellow'},
        {'name': 'FC1\n4096', 'color': 'lightblue'},
        {'name': 'FC2\n4096', 'color': 'lightblue'},
        {'name': 'FC3\n1000', 'color': 'lightblue'}
    ], "VGG19 is a deep convolutional network with 19 layers, featuring 5 blocks of convolutions with increasing filters, followed by three fully connected layers for classification."

def get_inceptionv3_blocks():
    return [
        {'name': 'Input\n299x299x3', 'color': 'lightblue'},
        {'name': 'Stem\n35x35x384', 'color': 'lightgreen'},
        {'name': 'Inception A\n35x35x256', 'color': 'lightcoral'},
        {'name': 'Inception B\n35x35x288', 'color': 'lightcoral'},
        {'name': 'Inception C\n35x35x288', 'color': 'lightcoral'},
        {'name': 'Reduction A\n17x17x768', 'color': 'lightyellow'},
        {'name': 'Inception D\n17x17x768', 'color': 'lightcoral'},
        {'name': 'Reduction B\n8x8x1280', 'color': 'lightyellow'},
        {'name': 'Inception E\n8x8x2048', 'color': 'lightcoral'},
        {'name': 'Global Avg Pool\n1x1x2048', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ], "InceptionV3 uses an inception module with parallel convolutions (1x1, 3x3, 5x5) and pooling, organized into Stem, multiple Inception stages, and Reduction layers, concluding with global pooling."

def get_mobilenetv2_blocks():
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Conv1\n112x112x32', 'color': 'lightgreen'},
        {'name': 'Bottleneck 1\n112x112x16', 'color': 'lightcoral'},
        {'name': 'Bottleneck 2\n56x56x24', 'color': 'lightcoral'},
        {'name': 'Bottleneck 3\n28x28x32', 'color': 'lightcoral'},
        {'name': 'Bottleneck 4\n14x14x64', 'color': 'lightcoral'},
        {'name': 'Bottleneck 5\n14x14x96', 'color': 'lightcoral'},
        {'name': 'Bottleneck 6\n7x7x160', 'color': 'lightcoral'},
        {'name': 'Bottleneck 7\n7x7x320', 'color': 'lightcoral'},
        {'name': 'Conv2\n7x7x1280', 'color': 'lightgreen'},
        {'name': 'Global Avg Pool\n1x1x1280', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ], "MobileNetV2 employs lightweight depthwise separable convolutions in bottleneck blocks, optimized for mobile devices with 17 bottlenecks and a final convolutional layer."

def get_densenet121_blocks():
    return [
        {'name': 'Input\n224x224x3', 'color': 'lightblue'},
        {'name': 'Conv1\n112x112x64', 'color': 'lightgreen'},
        {'name': 'Pool1\n56x56x64', 'color': 'lightpink'},
        {'name': 'Dense Block 1\n56x56x256', 'color': 'lightcoral'},
        {'name': 'Transition 1\n28x28x128', 'color': 'lightyellow'},
        {'name': 'Dense Block 2\n28x28x512', 'color': 'lightcoral'},
        {'name': 'Transition 2\n14x14x256', 'color': 'lightyellow'},
        {'name': 'Dense Block 3\n14x14x1024', 'color': 'lightcoral'},
        {'name': 'Transition 3\n7x7x512', 'color': 'lightyellow'},
        {'name': 'Dense Block 4\n7x7x1024', 'color': 'lightcoral'},
        {'name': 'Global Avg Pool\n1x1x1024', 'color': 'lightpink'},
        {'name': 'Fully Connected\n1000 classes', 'color': 'lightblue'}
    ], "DenseNet121 uses dense connectivity where each layer connects to all previous layers, with 4 dense blocks and transition layers for downsampling."

# Generate diagrams for all models
models_blocks = {
    'ResNet50': get_resnet50_blocks(),
    'VGG19': get_vgg19_blocks(),
    'InceptionV3': get_inceptionv3_blocks(),
    'MobileNetV2': get_mobilenetv2_blocks(),
    'DenseNet121': get_densenet121_blocks()
}

for model_name, (blocks, explanation) in models_blocks.items():
    output_file = f"{model_name.lower()}_architecture.png"
    plot_model_architecture(model_name, blocks, explanation, output_file=output_file)